In [6]:
from scipy.sparse import csr_matrix, block_diag
import cv2
import numpy as np
import pandas as pd

In [17]:
np.set_printoptions(precision=3, floatmode='fixed')

### №17

In [18]:
points_3d = np.random.rand(10, 3)
points_3d[:, 2] += 1

K = np.eye(3)

R1 = np.eye(3)
t1 = np.zeros((3, 1))

R2 = np.eye(3)
t2 = np.array([1, 0 ,0], dtype=np.float32)

points_2d_1, _ = cv2.projectPoints(points_3d, R1, t1, K, None)
points_2d_2, _ = cv2.projectPoints(points_3d, R2, t2, K, None)

E, _ = cv2.findEssentialMat(points_2d_1, points_2d_2, K, cv2.RANSAC)

print("Необходимая матрица E:")
print(E)
print("Сингулярные числа E:")
print(np.linalg.svd(E)[1])

Необходимая матрица E:
[[-2.591e-16 -8.527e-16  1.269e-15]
 [-9.941e-16  1.861e-15  7.071e-01]
 [ 2.432e-16 -7.071e-01 -1.333e-15]]
Сингулярные числа E:
[7.071e-01 7.071e-01 2.591e-16]


### №18

In [19]:
R1, R2, T = cv2.decomposeEssentialMat(E)
valid_poses = []
num_valid_points = []

for i in range(4):
    P1 = np.hstack((np.eye(3, 3), np.zeros((3, 1))))
    if i == 0:
        P2 = np.hstack((R1, T))
    elif i == 1:
        P2 = np.hstack((R1, -T))
    elif i == 2:
        P2 = np.hstack((R2, T))
    elif i == 3:
        P2 = np.hstack((R2, -T))

    points_2d_3, _ = cv2.projectPoints(points_3d, P1[:, :3], P1[:, 3:], K, None)
    points_2d_4, _ = cv2.projectPoints(points_3d, P2[:, :3], P2[:, 3:], K, None)

    if np.all(points_2d_3 > 0) and np.all(points_2d_4 > 0):
        print("Возможная поза первой камеры относительно второй [R|T]:")
        print(P2)

Возможная поза первой камеры относительно второй [R|T]:
[[ 1.000e+00  2.139e-15  2.612e-15  1.000e+00]
 [-2.627e-15  1.000e+00 -3.886e-16 -2.283e-15]
 [-2.548e-15  3.331e-16  1.000e+00 -1.142e-15]]
Возможная поза первой камеры относительно второй [R|T]:
[[ 1.000e+00 -1.451e-15  2.000e-16 -1.000e+00]
 [-1.939e-15 -1.000e+00  3.886e-16  2.283e-15]
 [ 2.642e-16 -3.331e-16 -1.000e+00  1.142e-15]]


### №19

In [11]:
image_path = "GOPR01170000.jpg"
image = cv2.imread(image_path)

f_len = 0.64 * 25.4
image_width = image.shape[1]
image_height = image.shape[0]

pixel_size_x = 36 / image_width
pixel_size_y = 24 / image_height

f_len_px = f_len / pixel_size_x
f_len_py = f_len / pixel_size_y

cx = image_width / 2
cy = image_height / 2

K1 = np.array([[4.2581151397691417e+02, 0.0, 4.8175802555527360e+02], 
               [ 0.0, 4.3533255745518812e+02, 2.6743704185634374e+02], 
               [ 0.0, 0.0, 1.0]], dtype=np.float32)

K2 = np.array([[f_len_px, 0, cx],
              [0, f_len_py, cy],
              [0, 0, 1]], dtype=np.float32)

print("Параметры, полученные методом калибрации камеры")
print(K1)
print("Параметры, полученные при предположении")
print(K2)

Параметры, полученные методом калибрации камеры
[[426.   0. 482.]
 [  0. 435. 267.]
 [  0.   0.   1.]]
Параметры, полученные при предположении
[[433.   0. 480.]
 [  0. 366. 270.]
 [  0.   0.   1.]]


### №20

In [8]:
np.set_printoptions(precision=0, floatmode='fixed')

In [9]:
n = 10
m = 3 * 10 ** 6

r = pd.read_csv('rhs.csv', header=None)
r = r.values.flatten()

i = np.arange(n).reshape(n, 1)
j = np.arange(n)
C = 1 / (i + j + 1)

i = np.arange(1, n + 1)
j = np.arange(1, m + 1)
I, J = np.meshgrid(i, j, indexing='ij')
D = np.cos(I * J)

Gb = np.array([[1, 0, 1],
              [0, 1, 0],
              [-1, 0, 1]])

Gb = [Gb] * (m // Gb.shape[0])
G = csr_matrix(block_diag(Gb))

Gb_inv = np.linalg.inv(Gb)
G_inv = csr_matrix(block_diag(Gb_inv))

A = C - D @ G_inv @ D.T

a = r[:10]
b = r[10:]

Delta_c = np.linalg.solve(A, a - D @ G_inv @ b)
Delta_S = G_inv @ (b - D.T @ Delta_c)

x = np.concatenate([Delta_c, Delta_S])

print("Первые 20 значений вектора x:", x[:20])

Первые 20 значений вектора x: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
